In [394]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import os
import implicit

In [380]:
data_path = 'data/'                         #Загружаем данные
data_1 = 'data_learn_match.csv'
data_2 = 'data_test_match.csv'
data_3 = 'data_target_thread.csv'
raw_data = pd.read_csv(os.path.join(data_path, data_1), header=None)
raw_data.drop(raw_data.columns[2:4], axis=1, inplace=True)
raw_data['values'] = 1
raw_data

,0,1,values
0,1,416,1
1,1,972,1
2,1,2732,1
3,1,3717,1
4,1,10455,1
...,...,...,...
9851234,59200,102756,1
9851235,59200,103592,1
9851236,59200,103734,1
9851237,59200,103886,1


In [381]:
raw_data.columns = ['user_uid', 'thread_uid', 'values']
data = raw_data.copy()

In [382]:
list_match_user = data['user_uid'].unique()  #создаем список уникальных пользователей

In [383]:
list_match_thread = data['thread_uid'].unique()  #создаем список уникальных складчин

In [384]:
raw_data_2 = pd.read_csv(os.path.join(data_path, data_2), header=None)
raw_data_2['values'] = 1

In [385]:
raw_data_2.columns = ['user_uid_test', 'thread_uid_test', 'values']
data_2 = raw_data_2.copy()
data_2

,user_uid_test,thread_uid_test,values
0,27601,103212,1
1,39239,103212,1
2,51088,103212,1
3,27601,103213,1
4,33457,103213,1
...,...,...,...
4952,52801,103648,1
4953,53678,103648,1
4954,54521,103648,1
4955,56488,103648,1


In [386]:
list_test_user = data_2['user_uid_test'].unique()  #аналогично, только на тестовом наборе

In [387]:
list_test_thread = data_2['thread_uid_test'].unique()

In [388]:
data_threads_test = pd.read_csv(os.path.join(data_path, data_3), header=None)
data_threads_test.columns = ['thread_uid_test']
data_threads_test

,thread_uid_test
0,103212
1,103213
2,103214
3,103215
4,103216
...,...
392,103644
393,103645
394,103646
395,103647


In [389]:
data_threads_test = data_threads_test['thread_uid_test']

In [390]:
data_threads_filter = list(set(list_match_thread).difference(data_threads_test)) #список для отсева неактуальных складчин

In [391]:
list_test_user.sort()

In [392]:
sparse_item_user = sparse.csr_matrix((data['values'].astype(float), (data['thread_uid'], data['user_uid'])))
sparse_user_item = sparse.csr_matrix((data['values'].astype(float), (data['user_uid'], data['thread_uid'])))

In [393]:
sparse_user_item.shape

(59202, 104060)

In [23]:
#СОбственно, сама модель и ее обучение. 
model = implicit.als.AlternatingLeastSquares(factors=2000, regularization=0.01, iterations=30)
alpha_val = 4
data_conf = (sparse_item_user*alpha_val).astype('double')
model.fit(data_conf)

In [329]:
#Рекомендации. Для каждого пользователя предсказываем 50 складчин (отфильтровав старые), затем фильтруем результаты по медиане.
rec = []
scores = []
for userid in list_test_user:
    recommended = model.recommend(userid, sparse_user_item, N=50, filter_already_liked_items = True, filter_items = data_threads_filter)
    for score in recommended:
        scores.append(score[1])
    median = np.median(scores)
    condition = lambda x: x[1] >= median
    filtered_recommended = list(filter(condition, recommended))
    recommended_1 = [x[0] for x in filtered_recommended]
    rec.append(recommended_1)
rec

[[103578,
  103542,
  103625,
  103269,
  103311,
  103509,
  103506,
  103315,
  103491,
  103314,
  103280,
  103604,
  103565,
  103427,
  103236,
  103465,
  103553,
  103505,
  103439,
  103255,
  103259,
  103431,
  103540,
  103303,
  103579],
 [103528,
  103514,
  103509,
  103552,
  103288,
  103225,
  103454,
  103505,
  103314,
  103526,
  103459,
  103642,
  103605,
  103575,
  103283,
  103445,
  103418,
  103463,
  103468,
  103533,
  103474,
  103532,
  103644,
  103330,
  103413,
  103521,
  103274,
  103224,
  103584,
  103254,
  103379,
  103592,
  103491,
  103631,
  103426,
  103485,
  103347,
  103629,
  103214,
  103300,
  103299],
 [103457,
  103515,
  103474,
  103556,
  103418,
  103601,
  103447,
  103242,
  103592,
  103302,
  103596,
  103530,
  103514,
  103481,
  103261,
  103641,
  103442,
  103553,
  103488,
  103427,
  103260,
  103241,
  103428,
  103439,
  103498,
  103294,
  103600,
  103550,
  103226,
  103510,
  103646,
  103476,
  103222,
  103373

Далее собственно обработка результатов, подсчет метрик. Для каждого юзера создается список складчин из тестового набора и список из предсказанных. Пересечение этих списков деленое на длину списка предсказаний = precision, деленое на длину списка из тестового набора = recall.

In [395]:
data_2['columns'] = data_2.groupby('user_uid_test')['thread_uid_test'].cumcount()

In [396]:
result = data_2.pivot(index='user_uid_test', columns='columns')

In [397]:
result[0:3]

thread_uid_test                                            ...  \
columns                    0         1   2   3   4   5   6   7   8   9   ...   
user_uid_test                                                            ...   
47                   103578.0       NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   
51                   103474.0       NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   
52                   103233.0  103506.0 NaN NaN NaN NaN NaN NaN NaN NaN  ...   

              values                                      
columns           43  44  45  46  47  48  49  50  51  52  
user_uid_test                                             
47               NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
51               NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
52               NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  

[3 rows x 106 columns]

In [398]:
result.drop(['values'], axis=1, inplace=True)

In [399]:
result['concat_col']=result.apply(lambda row: row.dropna().tolist(), axis=1)

In [400]:
result['concat_col']

user_uid_test
47                                     [103578.0]
51                                     [103474.0]
52                           [103233.0, 103506.0]
115                                    [103442.0]
156                          [103613.0, 103625.0]
                           ...                   
57232                                  [103634.0]
57242                                  [103594.0]
57261    [103224.0, 103285.0, 103438.0, 103541.0]
57274                                  [103345.0]
57287                                  [103516.0]
Name: concat_col, Length: 2717, dtype: object

In [401]:
df_test_threads = result[['concat_col']]
df_test_threads

,concat_col
columns,
user_uid_test,
47,[103578.0]
51,[103474.0]
52,"[103233.0, 103506.0]"
115,[103442.0]
156,"[103613.0, 103625.0]"
...,...
57232,[103634.0]
57242,[103594.0]


In [402]:
df_test_threads['predict'] = rec

C:\Users\Anton\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [403]:
df_test_threads

,concat_col,predict
columns,,
user_uid_test,,
47,[103578.0],"[103578, 103542, 103625, 103269, 103311, 10350..."
51,[103474.0],"[103528, 103514, 103509, 103552, 103288, 10322..."
52,"[103233.0, 103506.0]","[103457, 103515, 103474, 103556, 103418, 10360..."
115,[103442.0],"[103442, 103283, 103646, 103617, 103427, 10329..."
156,"[103613.0, 103625.0]","[103418, 103450, 103613, 103449, 103530, 10350..."
...,...,...
57232,[103634.0],[103603]
57242,[103594.0],"[103594, 103225, 103463, 103282, 103473, 10327..."


In [416]:
relev = []
for x in range(df_test_threads.shape[0]):
    A = list(set(df_test_threads['concat_col'].iloc[x]) & set(df_test_threads['predict'].iloc[x]))
    relev.append(len(A))

In [407]:
df_test_threads['relevant'] = relev

C:\Users\Anton\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [408]:
df_test_threads

,concat_col,predict,relevant
columns,,,
user_uid_test,,,
47,[103578.0],"[103578, 103542, 103625, 103269, 103311, 10350...",1
51,[103474.0],"[103528, 103514, 103509, 103552, 103288, 10322...",1
52,"[103233.0, 103506.0]","[103457, 103515, 103474, 103556, 103418, 10360...",0
115,[103442.0],"[103442, 103283, 103646, 103617, 103427, 10329...",1
156,"[103613.0, 103625.0]","[103418, 103450, 103613, 103449, 103530, 10350...",1
...,...,...,...
57232,[103634.0],[103603],0
57242,[103594.0],"[103594, 103225, 103463, 103282, 103473, 10327...",1


In [409]:
df_test_threads['len_concat'] = df_test_threads['concat_col'].apply(lambda x: len(x))
df_test_threads['len_predict'] = df_test_threads['predict'].apply(lambda x: len(x))

C:\Users\Anton\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Anton\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [410]:
df_test_threads['precision'] = df_test_threads['relevant']/df_test_threads['len_predict']
df_test_threads['recall'] = df_test_threads['relevant']/df_test_threads['len_concat']

C:\Users\Anton\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Anton\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [411]:
list_drop = list(set(list_test_user).difference(list_match_user))

In [412]:
list_drop

[50633, 52109, 56048, 50612]

In [413]:
df_test_threads_f = df_test_threads.drop(list_drop, axis = 0)
df_test_threads_f[0:15]

,concat_col,predict,relevant,len_concat,len_predict,precision,recall
columns,,,,,,,
user_uid_test,,,,,,,
47,[103578.0],"[103578, 103542, 103625, 103269, 103311, 10350...",1,1,25,0.040000,1.000000
51,[103474.0],"[103528, 103514, 103509, 103552, 103288, 10322...",1,1,41,0.024390,1.000000
52,"[103233.0, 103506.0]","[103457, 103515, 103474, 103556, 103418, 10360...",0,2,42,0.000000,0.000000
115,[103442.0],"[103442, 103283, 103646, 103617, 103427, 10329...",1,1,17,0.058824,1.000000
156,"[103613.0, 103625.0]","[103418, 103450, 103613, 103449, 103530, 10350...",1,2,24,0.041667,0.500000
171,"[103318.0, 103352.0, 103358.0, 103367.0, 10345...","[103538, 103434, 103459, 103549, 103446, 10327...",1,7,25,0.040000,0.142857
175,[103418.0],"[103457, 103553, 103625, 103215, 103601, 10363...",0,1,21,0.000000,0.000000
178,[103627.0],"[103243, 103622, 103327, 103557, 103331, 10355...",0,1,34,0.000000,0.000000


In [414]:
Precision = df_test_threads_f[['precision']].mean()
Precision

           columns
precision             0.063089
dtype: float64

In [415]:
Recall = df_test_threads_f[['recall']].mean()
Recall

        columns
recall             0.501025
dtype: float64